In [1]:
from helper import aoc_timer
import pandas as pd
import re
import itertools
from collections import deque

In [2]:
class Shop:
    """Class for item shop."""

    def __init__(self, path):
        self.inventory = self.get_shop(path)

    def get_shop(self, path):
        """Read shop inventory into pandas DataFrame."""
        df = []
        for line in open(path).read().split('\n'):
            if not line:
                continue
            if ':' in line:
                # New item type (plus headers)
                idx = line.find(':')
                item_type = line[:idx]
                headers = ['Type', 'Item'] + line[idx+1:].split()
            else:
                # Item row - use RegEx to split on 2+ whitespaces
                df.append({k: v for k, v in zip(headers, [item_type] + re.split(r'\s{2,}', line))})
        return pd.DataFrame(df).astype({k: 'int8' for k in headers[2:]})


In [3]:
class Spellbook:
    """Class for magic spells."""
    
    def __init__(self, path):
        self.spells = pd.read_csv(path)


In [4]:
class Game:
    """Class for battling two characters."""

    def __init__(self, characters):
        self.turn = 0
        self.characters = characters
        self.attacker, self.defender = characters
        self.battle_active = False
        self.battle_winner = None

    def __str__(self):
        """Print battle status."""
        lines = [
            f"Battle between:\n\n{self.attacker.__str__()}\n",
            "and\n",
            f"{self.defender.__str__()}\n",
            f"Active: {self.battle_active}",
            f"Turns: {self.turn}",
            f"Winner: {self.battle_winner.__class__.__name__}"
        ]
        return "\n".join(lines)

    def apply_effects(self, characters):
        """Apply active effects for a turn."""
        result = True
        for (char1, char2) in [characters, characters[::-1]]:
            if isinstance(char1, Wizard) and char1.active_effects:
                # Apply Damage, Armor, Mana over time effects
                for _, effect in char1.active_effects.items():
                    char2.hp = max(0, char2.hp - effect['dot'])
                    char1.armor += effect['aot']
                    char1.mana += effect['mot']
                    effect['timer'] -= 1
                # Pop expired effects
                char1.active_effects = {k: v for k, v in char1.active_effects.items() if v['timer'] > 0}
                if not char2.hp:
                    # char2 is the only character whose hp can reduce
                    self.battle_winner = char1
                    result = False
        return result

    def take_turn(self):
        """Apply effects then perform attack."""
        # Apply effects
        if not self.apply_effects([self.attacker, self.defender]):
            return False
        # Attacker attacks defender
        if not self.attacker.attack(self.defender):
            return False
        self.attacker, self.defender = self.defender, self.attacker
        self.turn += 1
###########################################################
#         print("After turn:")
#         print("-" * 80)
#         print(self.characters[0], "\n")
#         print(self.characters[1], "\n")
###########################################################
        # Reset Wizard attributes
        for char in self.characters:
            if isinstance(char, Wizard):
                char.reset_attribs()
        return True

    def battle(self, output=False):
        """Repeatedly attack until battle ends."""
        self.battle_active = True
        while self.battle_active:
###########################################################
#             print("\nBefore turn:")
#             print("-" * 80)
#             print(self.characters[0], "\n")
#             print(self.characters[1], "\n")
###########################################################
            self.battle_active = self.take_turn()
        # Battle complete - determine winner if not already determined
        if self.battle_winner is None:
            if self.characters[1].hp:
                self.battle_winner = self.characters[1]
            else:
                self.battle_winner = self.characters[0]
        # Return battle results
        if self.battle_winner == self.characters[0]:
            if output:
                print("You win!")
                print(self.characters[0])
            return True
        else:
            if output:
                print("You lose!")
                print(self.characters[0])
            return False


In [5]:
class Character:
    """Base class for RPG characters."""

    def __init__(self, hp=0, damage=0, armor=0, path=None):
        self._path = path  # Optionally set character attributes from input file
        if self._path is None:
            # Manually set attributes when initializing
            self.hp = hp
            self.damage = damage
            self.armor = armor
        else:
            # Set attributes from input file
            attr = self.get_attr(self._path)
            self.hp = attr.get('Hit Points', 0)
            self.damage = attr.get('Damage', 0)
            self.armor = attr.get('Armor', 0)

    def __str__(self):
        """Print base attributes."""
        attrs = {
            'HP': self.hp,
            'Damage': self.damage,
            'Armor': self.armor
        }
        return f"Class: {self.__class__.__name__}\nAttributes: {attrs}"

    def get_attr(self, path):
        """Optionally set character attributes from input file."""
        return {k: int(v) for k, v in
                [line.strip().split(': ') for line in open(path).readlines()]}

    def attack(self, char):
        """Attack given character, char."""
        hit = max(1, self.damage - char.armor)
        char.hp = max(0, char.hp - hit)
        return char.hp > 0


In [6]:
class Warrior(Character):
    """RPG character class that uses standard attacks during battle.
       Warrior can also purchase and equip items from the shop using gold.
       These items affect the Warrior's base attributes.
    """

    def __init__(self, hp=0, damage=0, armor=0, path=None):
        # Inherit base stats from Character class
        super().__init__(hp, damage, armor, path)
        # Warrior attributes
        self.gold = 0
        self.weapon_slots = range(1, 2)  # Must equip a weapon
        self.armor_slots = range(0, 2)   # Armor is optional
        self.ring_slots = range(0, 3)    # Maximum two rings
        # Warrior inventory
        self.weapon_equip = None
        self.armor_equip = None
        self.rings_equip = None

    def __str__(self):
        """Print attributes and inventory."""
        attrs = super().__str__()
        equip = {
            'Weapon': self.weapon_equip,
            'Armor': self.armor_equip,
            'Rings': self.rings_equip,
            'Cost': self.gold
        }
        return f"{attrs}\nInventory: {equip}"

    def purchase(self, items, shop):
        """Purchase items from the shop, amend attributes, record gold cost."""
        buy = shop.inventory.loc[items]
        self.damage = buy.Damage.sum()
        self.armor = buy.Armor.sum()
        self.gold = buy.Cost.sum()
        # Equip items
        self.weapon_equip = buy.Item.loc[buy.Type == 'Weapons'].tolist()
        self.armor_equip = buy.Item.loc[buy.Type == 'Armor'].tolist()
        self.rings_equip = buy.Item.loc[buy.Type == 'Rings'].tolist()


In [7]:
class Wizard(Character):
    """RPG character class that casts magic spells during battle.
       Wizard can cast spells from the spellbook using mana.
       If Wizard cannot afford to cast any spells, he loses.
    """

    def __init__(self, hp=0, damage=0, armor=0, mana=0, path=None):
        # Inherit base stats from Character class
        super().__init__(hp, damage, armor, path)
        # Wizard attributes
        self.mana = mana
        self._damage = damage       # initial damage
        self._armor = armor         # initial armor
        self.cum_mana_cost = 0
        self.spell_strategy = None
        self.spellbook = None
        # Spell effects
        self.active_effects = {}

    def __str__(self):
        """Print attributes and inventory."""
        attrs = super().__str__()
        equip = {
            'Mana': self.mana,
            'Strategy': self.spell_strategy,
            'Effects': self.active_effects
        }
        return f"{attrs}\nInventory: {equip}"

    def get_spell(self):
        """Get user input spell (index) as integer."""
        return int(input("Enter a spell to cast: "))
        
    def set_spell_strategy(self, strategy, spellbook):
        """Create deque from strategy if strategy is iterable."""
        try:
            iterator = iter(strategy)
        except TypeError:
            pass
        else:
            self.spell_strategy = deque(strategy)
        self.spellbook = spellbook

    def cast(self, spell):
        """Cast a spell from spellbook."""
        spell = self.spellbook.spells.loc[spell]
        # Check spell is castable
        if spell.Cost > self.mana:
            print(f"You can't afford to cast {spell.Spell}!  " +
                  f"It costs {spell.Cost} mana, you have {self.mana} mana.\n" +
                 "You lose!")
            return False
        # Check spell effect is not already active
        if spell.Spell in self.active_effects:
            print(f"You cannot cast {spell.Spell} as its effect is still active " +
                  f"with timer {self.active_effects[spell.Spell]['timer']}.\n" +
                 "You lose!")
            return False
        # Set immediate attributes from spell
        self.mana -= spell.Cost
        self.cum_mana_cost += spell.Cost
        self.damage = spell.Damage
        self.hp += spell.Heal
        # Set effects
        if spell.Duration:
            self.active_effects[spell.Spell] = {
                'timer': spell.Duration,
                'dot': spell.DoT,
                'aot': spell.AoT,
                'mot': spell.MoT
            }
        return True

    def reset_attribs(self):
        """Reset damage and armor attributes to starting values."""
        self.damage = self._damage
        self.armor = self._armor

    def attack(self, char):
        """Attack given character, char."""
        # Cast next spell from spell strategy
        if self.spell_strategy is None:
            spell = self.get_spell()
        elif self.spell_strategy:
            spell = self.spell_strategy.popleft()
        else:
            return False
        if not self.cast(spell):
            return False
        # Apply instant damage from base class
        if self.damage:
            return super().attack(char)
        return True


In [8]:
@aoc_timer
def Day21(hp=100, output=False, part2=False):
    # Setup
    player = Warrior(hp, 0, 0)
    shop = Shop('shop.txt')

    # Combinations of available items
    inv = shop.inventory
    item_types = inv.Type.unique()
    combs = {k: [] for k in item_types}
    items = zip(
        [player.weapon_slots, player.armor_slots, player.ring_slots],  # Player inventory slots
        [inv.loc[inv.Type == x].index for x in item_types],            # Shop item indices
        item_types                                                     # Types of items
    )

    # Compile dictionary of all combinations of each item
    for slots, idx, item in items:
        for n in slots:
            for c in itertools.combinations(idx, n):
                combs[item].append(c)

    # Compile combinations of all possible items
    costs = {
        tuple(loadout): inv.Cost.loc[loadout].sum() for loadout in
        [list(sum(x, ())) for x in itertools.product(*combs.values())]
    }

    # Sort items by cost, purchase item combination and play game until condition is met
    for items, cost in sorted(costs.items(), key=lambda x: x[1], reverse=part2):
        # Reset characters
        player = Warrior(hp, 0, 0)
        boss = Character(path='day21.txt')
        characters = [player, boss]
        game = Game(characters)
        # Purchase item combination and battle
        player.purchase(list(items), shop)
        if game.battle(output) ^ part2:
            return cost

print("Part 1:", Day21(part2=False))
print("Part 2:", Day21(part2=True))

-----
Time: 329.0 ms
Part 1: 111
-----
Time: 416.0 ms
Part 2: 188


In [9]:
shop = Shop('shop.txt')
inv = shop.inventory
inv

,Type,Item,Cost,Damage,Armor
0,Weapons,Dagger,8,4,0
1,Weapons,Shortsword,10,5,0
2,Weapons,Warhammer,25,6,0
3,Weapons,Longsword,40,7,0
4,Weapons,Greataxe,74,8,0
5,Armor,Leather,13,0,1
6,Armor,Chainmail,31,0,2
7,Armor,Splintmail,53,0,3
8,Armor,Bandedmail,75,0,4
9,Armor,Platemail,102,0,5


In [10]:
spellbook = Spellbook('spells.txt')
spells = spellbook.spells
spells

,Spell,Cost,Damage,Heal,Duration,DoT,AoT,MoT
0,Magic Missile,53,4,0,0,0,0,0
1,Drain,73,2,2,0,0,0,0
2,Shield,113,0,0,6,0,7,0
3,Poison,173,0,0,6,3,0,0
4,Recharge,229,0,0,5,0,0,101


In [11]:
# Testing functionality
shop = Shop('shop.txt')
warrior = Warrior(hp=100)
first_boss = Character(path='day21.txt')
game1 = Game([warrior, first_boss])
spellbook = Spellbook('spells.txt')
wizard = Wizard(hp=50, mana=500)
second_boss = Character(path='input.txt')
game2 = Game([wizard, second_boss])

characters = [
    warrior,
    first_boss,
    wizard,
    second_boss
]

for char in characters:
    print(char)
    print()

print("=" * 80)

warrior.purchase([3, 6, 14], shop)
game1.battle()
print()
print(game1)
print()

Class: Warrior
Attributes: {'HP': 100, 'Damage': 0, 'Armor': 0}
Inventory: {'Weapon': None, 'Armor': None, 'Rings': None, 'Cost': 0}

Class: Character
Attributes: {'HP': 109, 'Damage': 8, 'Armor': 2}

Class: Wizard
Attributes: {'HP': 50, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 500, 'Strategy': None, 'Effects': {}}

Class: Character
Attributes: {'HP': 51, 'Damage': 9, 'Armor': 0}


Battle between:

Class: Warrior
Attributes: {'HP': 16, 'Damage': 7, 'Armor': 4}
Inventory: {'Weapon': ['Longsword'], 'Armor': ['Chainmail'], 'Rings': ['Defense +2'], 'Cost': 111}

and

Class: Character
Attributes: {'HP': 0, 'Damage': 8, 'Armor': 2}

Active: False
Turns: 42
Winner: Warrior



In [12]:
spellbook = Spellbook('spells.txt')
wizard = Wizard(hp=10, mana=250)
second_boss = Character(hp=13, damage=8)
game2 = Game([wizard, second_boss])
wizard.set_spell_strategy([3, 0], spellbook)
game2.battle(output=True)
print()
print(game2)


"""
For example, suppose the player has 10 hit points and 250 mana, and that the boss has 13 hit points and 8 damage:

-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --
- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage.

-- Player turn --
- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
"""

print()

You win!
Class: Wizard
Attributes: {'HP': 2, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 24, 'Strategy': deque([]), 'Effects': {'Poison': {'timer': 3, 'dot': 3, 'aot': 0, 'mot': 0}}}

Battle between:

Class: Character
Attributes: {'HP': 0, 'Damage': 8, 'Armor': 0}

and

Class: Wizard
Attributes: {'HP': 2, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 24, 'Strategy': deque([]), 'Effects': {'Poison': {'timer': 3, 'dot': 3, 'aot': 0, 'mot': 0}}}

Active: False
Turns: 3
Winner: Wizard



In [13]:
spellbook = Spellbook('spells.txt')
wizard = Wizard(hp=10, mana=250)
second_boss = Character(hp=14, damage=8)
game2 = Game([wizard, second_boss])
wizard.set_spell_strategy([4, 2, 1, 3, 0], spellbook)
game2.battle(output=True)
print()
print(game2)


"""
-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 14 hit points
Player casts Recharge.

-- Boss turn --
- Player has 10 hit points, 0 armor, 21 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 4.
Boss attacks for 8 damage!

-- Player turn --
- Player has 2 hit points, 0 armor, 122 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 3.
Player casts Shield, increasing armor by 7.

-- Boss turn --
- Player has 2 hit points, 7 armor, 110 mana
- Boss has 14 hit points
Shield's timer is now 5.
Recharge provides 101 mana; its timer is now 2.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 211 mana
- Boss has 14 hit points
Shield's timer is now 4.
Recharge provides 101 mana; its timer is now 1.
Player casts Drain, dealing 2 damage, and healing 2 hit points.

-- Boss turn --
- Player has 3 hit points, 7 armor, 239 mana
- Boss has 12 hit points
Shield's timer is now 3.
Recharge provides 101 mana; its timer is now 0.
Recharge wears off.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 2 hit points, 7 armor, 340 mana
- Boss has 12 hit points
Shield's timer is now 2.
Player casts Poison.

-- Boss turn --
- Player has 2 hit points, 7 armor, 167 mana
- Boss has 12 hit points
Shield's timer is now 1.
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 167 mana
- Boss has 9 hit points
Shield's timer is now 0.
Shield wears off, decreasing armor by 7.
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 1 hit point, 0 armor, 114 mana
- Boss has 2 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
"""

print()

You win!
Class: Wizard
Attributes: {'HP': 1, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 114, 'Strategy': deque([]), 'Effects': {'Poison': {'timer': 3, 'dot': 3, 'aot': 0, 'mot': 0}}}

Battle between:

Class: Character
Attributes: {'HP': 0, 'Damage': 8, 'Armor': 0}

and

Class: Wizard
Attributes: {'HP': 1, 'Damage': 0, 'Armor': 0}
Inventory: {'Mana': 114, 'Strategy': deque([]), 'Effects': {'Poison': {'timer': 3, 'dot': 3, 'aot': 0, 'mot': 0}}}

Active: False
Turns: 9
Winner: Wizard



In [15]:
@aoc_timer
def Day22(hp=50, mana=500, output=False, part2=False):
    # Setup
    player = Wizard(hp=hp, mana=mana)
    spellbook = Spellbook('spells.txt')

    # Combinations of castable spells
    spells = spellbook.spells
    max_spells = 10
    combs = itertools.combinations_with_replacement(spells.index, max_spells)

    for strat in combs:
        # Reset game
        player = Wizard(hp=hp, mana=mana)
        boss = Character(path='input.txt')
        characters = [player, boss]
        game = Game(characters)
        # Set spell strategy
        player.set_spell_strategy(list(strat), spellbook)
        if game.battle(output):
            return player.cum_mana_cost

# I don't think this is handling win conditions properly
# Double check return values for all battle ending conditions
# Currently returning False in various places to signify battle ending
# But I don't think winner is being set correctly everywhere

print("Part 1:", Day22(output=False, part2=False))
# print("Part 2:", Day22(part2=True))

You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 122 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 122 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 122 

You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!

You cannot cast Shield as its effect is still active with timer 4.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 15 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 15 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 15 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 15 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 15 mana.
You lose!
You can't afford to cast Recharge!  It costs 229 mana, you have 15 mana.
You lose!
You can't afford to cast Recharge!  It costs 229 mana, you have 188 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 128 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 128 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 128 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 128 mana.
You lose!
You can't afford to cas

You can't afford to cast Shield!  It costs 113 mana, you have 22 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 22 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 22 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 22 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 22 mana.
You lose!
You can't afford to cast Recharge!  It costs 229 mana, you have 22 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 135 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 135 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 135 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 135 mana.
You lose!
You can't afford to cast Poison!  It costs 173 mana, you have 135 mana.
You lose!
You can't afford to cast Recharge!  It costs 229 mana, you have 135 mana.
You lose!
You can't afford t

You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Shield as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!
You cannot cast Poison as its effect is still active with timer 4.
You lose!

In [47]:
test = pd.to_datetime('2021/02/14 17:00:00')
flight = pd.to_datetime('2021/02/17 09:00:00')
check = pd.to_datetime('2021/02/17 06:00:00')
to_flight = (flight - test).total_seconds()//3600
to_check = (check - test).total_seconds()//3600
print(f"To check-in: {to_check} hours\nTo flight: {to_flight} hours")

To check-in: 61.0 hours
To flight: 64.0 hours
